In [14]:
import argparse
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
import lightgbm as lgb
from sklearn.metrics import f1_score
import numpy as np

In [2]:
CATEGORY = ['ecfg', 'flbmk', 'flg_3dsmk', 'insfg', 'ovrlt', 'scity', 'csmcu', 'cano', 'mchno', 'hcefg', 'bacno', 'contp', 'etymd', 'acqic']

In [3]:
def lgb_f1_score(y_true, y_pred):
    y_hat = np.round(y_pred)
    return 'f1', f1_score(y_true, y_hat), True

In [4]:
df_train = pd.read_csv("train.csv")

In [5]:
df_train.head()

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6881,113261,38038,513.80,5,0,N,0,N,N,...,0,33,172652.0,457,59333,N,0,102,0,516056
1,0,134508,45725,465.62,5,0,N,2,N,N,...,0,9,105114.0,451,0,N,5817,102,0,4376
2,6881,15408,188328,513.80,5,0,N,0,N,N,...,0,6,152458.0,457,59333,N,0,102,0,483434
3,6716,157159,29967,1016.11,5,62,N,5,N,N,...,0,5,172946.0,247,50436,N,3281,102,0,1407164
4,5975,105985,81305,713.66,5,62,N,4,N,N,...,0,6,182129.0,263,93775,N,5817,102,0,1051004


In [6]:
for cat in CATEGORY:
    df_train[cat] = df_train[cat].astype('category')#.cat.codes

In [7]:
y_train = df_train['fraud_ind']
x_train = df_train.drop('fraud_ind', axis=1) #直接drop draud 那欄 (axis = 1 ,drop column納欄)

In [8]:
x_train

,acqic,bacno,cano,conam,contp,csmcu,ecfg,etymd,flbmk,flg_3dsmk,...,iterm,locdt,loctm,mcc,mchno,ovrlt,scity,stocn,stscd,txkey
0,6881,113261,38038,513.80,5,0,N,0,N,N,...,0,33,172652.0,457,59333,N,0,102,0,516056
1,0,134508,45725,465.62,5,0,N,2,N,N,...,0,9,105114.0,451,0,N,5817,102,0,4376
2,6881,15408,188328,513.80,5,0,N,0,N,N,...,0,6,152458.0,457,59333,N,0,102,0,483434
3,6716,157159,29967,1016.11,5,62,N,5,N,N,...,0,5,172946.0,247,50436,N,3281,102,0,1407164
4,5975,105985,81305,713.66,5,62,N,4,N,N,...,0,6,182129.0,263,93775,N,5817,102,0,1051004
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1521782,6322,91008,15189,578.38,5,75,Y,8,NaN,NaN,...,0,4,191642.0,209,38222,N,5817,102,0,1478280
1521783,3226,145107,116252,435.32,5,75,Y,8,NaN,NaN,...,0,13,102338.0,192,90135,N,1458,102,0,661087
1521784,6769,162168,93598,1.38,5,75,Y,8,NaN,NaN,...,0,29,234618.0,373,79246,N,5817,102,0,167073
1521785,6032,45406,197460,1.38,5,75,Y,2,NaN,NaN,...,0,24,215218.0,373,79246,N,5817,102,0,338215


In [9]:
x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2)

In [10]:
x_test.shape

(304358, 22)

In [11]:
x_train.shape

(1217429, 22)

In [12]:
estimator = lgb.LGBMClassifier(num_leaves=31)

In [15]:
param_grid = {
        'learning_rate': [0.1],
        'n_estimators': [1000],
        'scale_pos_weight': [3, 5, 70, 100]
        }



In [16]:
gbm = GridSearchCV(estimator, 
                   param_grid, 
                   cv = 10,
                   scoring='f1', 
                   return_train_score = True,
                   n_jobs = -1)
                                                                                #連續五次迭次就停止

In [18]:
def main(args):
    df_train = pd.read_csv(args.train_file)

    for cat in CATEGORY:
        df_train[cat] = df_train[cat].astype('category')#.cat.codes

    y_train = df_train['fraud_ind']
    x_train = df_train.drop('fraud_ind', axis=1) #drop y 

    x_train, x_test, y_train, y_test = train_test_split(x_train, y_train, test_size=0.2)

    estimator = lgb.LGBMClassifier(num_leaves=31)

    param_grid = {
            'learning_rate': [0.1],
            'n_estimators': [1000],
            'scale_pos_weight': [3, 5, 70, 100]
            }

    gbm = GridSearchCV(estimator, 
                       param_grid, 
                       cv = 10,
                       scoring='f1', 
                       return_train_score = True,
                       n_jobs = -1)
    gbm.fit(x_train, y_train, eval_set=[(x_test, y_test)], eval_metric=lgb_f1_score, early_stopping_rounds=5, categorical_feature='auto')
                                                                                     #連續五次迭次就停止
    # cross-validation result
    df = pd.DataFrame(gbm.cv_results_)
    df.sort_values(by = "mean_test_score", inplace =True)
    df.to_csv(args.logs_path, index=False)
    mean_test_score = df.iloc[0].mean_test_score
    std_test_score = df.iloc[0].std_test_score
    print ("10-fold validating result on best paras : {} with +/- {}".format(round(mean_test_score, 4), round(std_test_score,4)))
    
    # loading testing data 
    df_test = pd.read_csv(args.test_file)
    for cat in CATEGORY:
        df_test[cat] = df_test[cat].astype('category')

    # prediction
    result = gbm.predict(df_test)
    df_label = pd.DataFrame(result, columns=['fraud_ind'])
    df = pd.merge(df_test, df_label, left_index=True, right_index=True)
    df[['txkey', 'fraud_ind']].to_csv(args.result_path, index=False)

In [21]:
import numpy as np
from sklearn.model_selection import KFold


In [25]:
X = ["a", "b", "c", "d"]
kf = KFold(n_splits=2)
for train, test in kf.split(X):
    print("%s %s" % (train, test))


[2 3] [0 1]
[0 1] [2 3]


In [26]:
kf

KFold(n_splits=2, random_state=None, shuffle=False)

In [27]:
kf.split(X)

<generator object _BaseKFold.split at 0x1259fe620>

In [28]:
X = np.array([[0., 0.], [1., 1.], [-1., -1.], [2., 2.]])
y = np.array([0, 1, 0, 1])
X_train, X_test, y_train, y_test = X[train], X[test], y[train], y[test]


In [29]:
X

array([[ 0.,  0.],
       [ 1.,  1.],
       [-1., -1.],
       [ 2.,  2.]])

In [35]:
X[test]

array([[-1., -1.],
       [ 2.,  2.]])

In [32]:
X_train

array([[0., 0.],
       [1., 1.]])

In [33]:
train

array([0, 1])

In [34]:
test

array([2, 3])

# -------------------------------------------------------------

In [ ]:
def group_target_by_cols(df_train, df_test, recipe):
    df = pd.concat([df_train, df_test], axis = 0)
    for m in range(len(recipe)):
        cols = recipe[m][0]
        for n in range(len(recipe[m][1])):
            target = recipe[m][1][n][0]
            method = recipe[m][1][n][1]
            name_grouped_target = method+"_"+target+'_BY_'+'_'.join(cols)
            tmp = df[cols + [target]].groupby(cols).agg(method)
            tmp = tmp.reset_index().rename(index=str, columns={target: name_grouped_target})
            df_train = df_train.merge(tmp, how='left', on=cols)
            df_test = df_test.merge(tmp, how='left', on=cols)

        # reduced memory    
        del tmp
        gc.collect()
    
    return df_train, df_test